# 库

In [1]:
import numpy as np
import mnist 
np.random.seed(114514)  #随机数种子

# 卷积核(滤波器)类

In [2]:
class Conv3x3:
    #3*3的卷积层
    
    def __init__(self, num_filters):
        
        self.num_filters = num_filters
        #num_filters即为滤波器的数量
        
        self.filters = np.random.uniform(-1, 1, (num_filters, 3, 3))
        #将滤波器权重初始化为均匀分布的值（矩阵中的每个元素大小都在-1到1之间）
        
    def iterate_regions(self, image):
        #设定迭代区域
        
        h, w = image.shape
        #h表示图像高度，w表示宽度
        
        for i in range(h - 2):
            for j in range(w - 2):
                im_region = image[i:(i + 3), j:(j + 3)]
                # 生成图像的所有可能的 3x3 子区域
                
                yield im_region, i, j
                #将子区域的信息存入迭代器
 
    def forward(self, input):
        #向前传播函数
        
        self.last_input = input
        
        h, w = input.shape
        #mnist数据集的图片大小为28*28,因此input也是28*28
        
        output = np.zeros((h - 2, w - 2, self.num_filters))
        #output为26*26*num_filters
 
        for im_region, i, j in self.iterate_regions(input):
        
            # 卷积运算，点乘再相加，ouput[i, j] 为向量，8 层
            output[i, j] = np.sum(im_region * self.filters, axis=(1, 2))
            
        # 最后将输出数据返回，便于下一层的输入使用
        return output
    
    def backprop(self, d_L_d_out, learn_rate):
        
        # 初始化一组为 0 的 gradient，3x3x8
        d_L_d_filters = np.zeros(self.filters.shape)
 
        # im_region，一个个 3x3 小矩阵
        for im_region, i, j in self.iterate_regions(self.last_input):
            for f in range(self.num_filters):
                # 按 f 分层计算，一次算一层，然后累加起来
                # d_L_d_filters[f]: 3x3滤波器
                # d_L_d_out[i, j, f]: num
                # im_region: 图片中3*3的区域
                d_L_d_filters[f] += d_L_d_out[i, j, f] * im_region
 
        # 更新滤波器
        self.filters -= learn_rate * d_L_d_filters
 
        return None

# 池化层类

In [3]:
class MaxPool2:
    # 2*2的池化层
    
    def iterate_regions(self,image):
        #此时的image为26*26*num_filters的三维数组
        
        h,w,_=image.shape
        # _表示不获取它的深度，即有几层
        
        new_h = h//2 #类似C语言的 h/2
        new_w = w//2
        
        for i in range(new_h):
            for j in range(new_w):
                im_region = image[(i*2):(i*2+2),(j*2):(j*2+2)]
                yield im_region,i,j
        #上面是和卷积核类同样的切片操作
        
    def forward(self,input):
        
        self.last_input = input
        
        h,w,num_filters = input.shape
        #获取image的三围
        
        output = np.zeros((h//2,w//2,num_filters))
        
        for im_region,i,j in self.iterate_regions(input):
            output[i,j] = np.amax(im_region,axis=(0,1))
            #计算之前切片操作的那个2*2矩阵里的元素的最大值
            
        return output 
    
    def backprop(self, d_L_d_out):
        
        # 池化层输入数据，26x26x8，默认初始化为 0
        d_L_d_input = np.zeros(self.last_input.shape)
 
        # 每一个 im_region 都是一个 3x3x8 的8层小矩阵
        # 修改 max 的部分，首先查找 max
        for im_region, i, j in self.iterate_regions(self.last_input):
            h, w, f = im_region.shape
            # 获取 im_region 里面最大值的索引向量，一叠的感觉
            amax = np.amax(im_region, axis=(0, 1))
 
            # 遍历整个 im_region，对于传递下去的像素点，修改 gradient 为 loss 对 output 的gradient
            for i2 in range(h):
                for j2 in range(w):
                    for f2 in range(f):
                        # 如果这个像素是最大值，则将其梯度复制到该像素
                        if im_region[i2, j2, f2] == amax[f2]:
                            d_L_d_input[i * 2 + i2, j * 2 + j2, f2] = d_L_d_out[i, j, f2]
 
        return d_L_d_input

# 全连接层类

In [4]:
class Softmax:
    # 用Softmax函数激活的全连接层
 
    def __init__(self, input_len, nodes):
        # input_len为输入层的节点个数(池化层处理之后的)
        # nodes为输出层的节点个数，因为需要识别数字0到9，因此结点个数为10
        
        self.weights = np.random.randn(input_len, nodes) / input_len
        # 构建权重矩阵，初始化随机数，不能太大
        #除以input_len以减少初始值的方差
        
        self.biases = np.zeros(nodes)
        #偏置向量用于在计算每个类别的得分时添加一个偏移量
        
 
    def forward(self , input):
        
        self.last_input_shape = input.shape
        # 保存所需数据，用于反向传播
        # 13*13*num_filters

        input = input.flatten()
        #三维数组降成一维数组，用于构建全连接层
        #例：[[1, 2, 3]       拉平后：[1, 2, 3, 4, 5, 6]    
        #     [4, 5, 6]]
        
        self.last_input = input
        # 一维向量 169*num_filters
 
        input_len, nodes = self.weights.shape
 
        totals = np.dot(input, self.weights) + self.biases
        #将输入数据 input 与权重矩阵 self.weights 进行点积操作，然后将得到的每个类别的得分加上相应的偏置，得到最终的 totals 数组。这个 totals 数组代表了每个类别的得分
        
        self.last_totals = totals
        #进行运算前的一维向量 10
        
        exp = np.exp(totals)
        
        return exp / np.sum(exp, axis=0)
        #softmax函数计算
    
    
    
    def backprop(self, d_L_d_out, learn_rate):
        #d_L_d_out是上一层输出对当前层输出的损失梯度，代表了当前层（即 Softmax 层）的输出对整个网络预测的贡献
        #learn_rate用于控制权重和偏置的更新
        
        # 已经知道在整个l_g_out中只有一个数不是0
        for i, gradient in enumerate(d_L_d_out):
            if gradient == 0:
                continue
 
            # e^totals
            t_exp = np.exp(self.last_totals)
 
            # 计算S
            S = np.sum(t_exp)
 
            # out[i]相对于总数的梯度
            d_out_d_t = -t_exp[i] * t_exp / (S ** 2)
            d_out_d_t[i] = t_exp[i] * (S - t_exp[i]) / (S ** 2)
 
            # 总量相对于权重/偏差/输入的梯度
            d_t_d_w = self.last_input
            d_t_d_b = 1
            d_t_d_inputs = self.weights
 
            # 损失相对于总量的梯度
            d_L_d_t = gradient * d_out_d_t
 
            # 相对于权重/偏差/输入的损失梯度
            d_L_d_w = d_t_d_w[np.newaxis].T @ d_L_d_t[np.newaxis]
            d_L_d_b = d_L_d_t * d_t_d_b
            d_L_d_inputs = d_t_d_inputs @ d_L_d_t
 
            #更新权重矩阵和偏置矩阵
            self.weights -= learn_rate * d_L_d_w
            self.biases -= learn_rate * d_L_d_b
            # 将矩阵从 1d 转为 3d
            # 169num_filters to 13x13xnum_filters
            return d_L_d_inputs.reshape(self.last_input_shape)

# 只有前向传播

In [5]:
test_images = mnist.test_images()[:1000]
test_labels = mnist.test_labels()[:1000]
 
conv = Conv3x3(8)                                    
pool = MaxPool2()                                    
softmax = Softmax(13 * 13 * 8, 10) 
 
def forward(image, label):
   
    out = conv.forward((image / 255) - 0.5)
    out = pool.forward(out)
    out = softmax.forward(out)
    #向前传播的过程
    
    loss = -np.log(out[label])
    
    acc = 1 if np.argmax(out) == label else 0
    # np.argmax(out)是概率最大的那个值的索引
    # 如果 softmax 输出的最大值就是 label 的值，表示正确，否则错误
 
    return out, loss, acc
 
print('MNIST数据集上的CNN初始化成功')
 
loss = 0
num_correct = 0

for i, (im, label) in enumerate(zip(test_images, test_labels)):

    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc
 
    if i % 100 == 99:
        print(
            '[Step %d] 过去一百张图片: 平均损失值 %.3f | 准确率: %d%%' %
            (i + 1, loss / 100, num_correct)
        )
        loss = 0
        num_correct = 0

MNIST数据集上的CNN初始化成功
[Step 100] 过去一百张图片: 平均损失值 2.302 | 准确率: 11%
[Step 200] 过去一百张图片: 平均损失值 2.304 | 准确率: 6%
[Step 300] 过去一百张图片: 平均损失值 2.302 | 准确率: 8%
[Step 400] 过去一百张图片: 平均损失值 2.302 | 准确率: 12%
[Step 500] 过去一百张图片: 平均损失值 2.303 | 准确率: 8%
[Step 600] 过去一百张图片: 平均损失值 2.301 | 准确率: 19%
[Step 700] 过去一百张图片: 平均损失值 2.302 | 准确率: 9%
[Step 800] 过去一百张图片: 平均损失值 2.303 | 准确率: 6%
[Step 900] 过去一百张图片: 平均损失值 2.300 | 准确率: 15%
[Step 1000] 过去一百张图片: 平均损失值 2.300 | 准确率: 12%


# 训练过后的CNN

In [6]:
train_images = mnist.train_images()[:1000]
train_labels = mnist.train_labels()[:1000]
test_images = mnist.test_images()[:1000]
test_labels = mnist.test_labels()[:1000]
 
conv = Conv3x3(8)                                    # 28x28x1 -> 26x26x8
pool = MaxPool2()                                    # 26x26x8 -> 13x13x8
softmax = Softmax(13 * 13 * 8, 10) # 13x13x8 -> 10
 
def forward(image, label):
    
    # We transform the image from [0, 255] to [-0.5, 0.5] to make it easier
    # to work with. This is standard practice.
    out = conv.forward((image / 255) - 0.5)
    out = pool.forward(out)
    out = softmax.forward(out)
 
    # Calculate cross-entropy loss and accuracy. np.log() is the natural log.
    loss = -np.log(out[label])
    acc = 1 if np.argmax(out) == label else 0
 
    return out, loss, acc
    # out: vertor of probability
    # loss: num
    # acc: 1 or 0
 
def train(im, label, lr=.005):
    
    # 前向传播
    out, loss, acc = forward(im, label)
 
    # 计算最开始的梯度（反向传播的开始）
    gradient = np.zeros(10)
    gradient[label] = -1 / out[label]
 
    # 反向传播
    gradient = softmax.backprop(gradient, lr)
    gradient = pool.backprop(gradient)
    gradient = conv.backprop(gradient, lr)
 
    return loss, acc
 
print('MNIST数据集上的CNN初始化成功')
 
#三轮迭代
for epoch in range(3):
    print('--- 第 %d 轮迭代---' % (epoch + 1))
 
    # Shuffle the training data
    permutation = np.random.permutation(len(train_images))
    train_images = train_images[permutation]
    train_labels = train_labels[permutation]
 
    # Train!
    loss = 0
    num_correct = 0
 
    # i为索引
    # im为图片
    # label为标签
    for i, (im, label) in enumerate(zip(train_images, train_labels)):
        if i > 0 and i % 100 == 99:
            print(
                '[Step %d] 过去100张图片: 平均损失值: %.3f | 准确率: %d%%' %
                (i + 1, loss / 100, num_correct)
            )
            loss = 0
            num_correct = 0
 
        l, acc = train(im, label)
        loss += l
        num_correct += acc
 
#最终测试
print('\n--- CNN的测试结果 ---')
loss = 0
num_correct = 0
for im, label in zip(test_images, test_labels):
    _, l, acc = forward(im, label)
    loss += l
    num_correct += acc
 
num_tests = len(test_images)
print('总体平均损失值:', loss / num_tests)
print('总体平均准确率:', num_correct / num_tests)

MNIST数据集上的CNN初始化成功
--- 第 1 轮迭代---
[Step 100] 过去100张图片: 平均损失值: 1.927 | 准确率: 43%
[Step 200] 过去100张图片: 平均损失值: 1.134 | 准确率: 69%
[Step 300] 过去100张图片: 平均损失值: 0.845 | 准确率: 71%
[Step 400] 过去100张图片: 平均损失值: 0.694 | 准确率: 79%
[Step 500] 过去100张图片: 平均损失值: 0.798 | 准确率: 76%
[Step 600] 过去100张图片: 平均损失值: 0.601 | 准确率: 82%
[Step 700] 过去100张图片: 平均损失值: 0.636 | 准确率: 81%
[Step 800] 过去100张图片: 平均损失值: 0.696 | 准确率: 80%
[Step 900] 过去100张图片: 平均损失值: 0.583 | 准确率: 83%
[Step 1000] 过去100张图片: 平均损失值: 0.441 | 准确率: 90%
--- 第 2 轮迭代---
[Step 100] 过去100张图片: 平均损失值: 0.356 | 准确率: 89%
[Step 200] 过去100张图片: 平均损失值: 0.317 | 准确率: 94%
[Step 300] 过去100张图片: 平均损失值: 0.542 | 准确率: 84%
[Step 400] 过去100张图片: 平均损失值: 0.495 | 准确率: 87%
[Step 500] 过去100张图片: 平均损失值: 0.352 | 准确率: 88%
[Step 600] 过去100张图片: 平均损失值: 0.246 | 准确率: 95%
[Step 700] 过去100张图片: 平均损失值: 0.524 | 准确率: 86%
[Step 800] 过去100张图片: 平均损失值: 0.475 | 准确率: 88%
[Step 900] 过去100张图片: 平均损失值: 0.240 | 准确率: 92%
[Step 1000] 过去100张图片: 平均损失值: 0.237 | 准确率: 94%
--- 第 3 轮迭代---
[Step 100] 过去100张图片: 平均损失值: 0.296 